In [2]:
from threading import Lock
import warnings
import matplotlib.pyplot as plt
import pyximport
from tqdm import tnrange, tqdm_notebook

warnings.filterwarnings('ignore')

import sys
import os

sys.path.insert(0, os.path.join(os.getcwd(), os.pardir))
from qutip import *
import numpy as np
import matplotlib.pyplot as plt

pyximport.install(setup_args={"include_dirs": np.get_include()})
import XXZZham as XXZZham
from XXZZham import add_high_energies, rotate_to_00_base
import random
import adiabatic_sim as asim
import time

import multiprocessing
import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
mkl_get_max_threads = mkl_rt.mkl_get_max_threads
mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(multiprocessing.cpu_count())))
import os
import LH_tools as LHT

%precision %.15f
PRECISION  = 2 ** -40

from IPython.display import display
import FD_tools as FDT

In [3]:
def _simulate_time_p_and_pfab_repeat(repeats: int, T: float, steps: int, psi0: Qobj, psi1: Qobj, H0: Qobj, H1: Qobj):

    """
        Callback for simulation of fowrard and backword propagation
    :param T:
    :param steps:
    :param psi0:
    :param psi1:
    :param H0:
    :param H1:
    :return:
    """
    tlist = np.linspace(0, T, steps)
    backprop_t, prop_t = FDT.create_back_and_forward_props(tlist, H0, H1)
    psi_forward = prop_t * psi0
    
    psi_repeat = (backprop_t*prop_t)**repeats * psi0
    
    Pf0 = abs(psi1.overlap(psi_forward)) ** 2
    Prepeat = abs(psi0.overlap(psi_repeat)) ** 2
    return T, Pf0, Prepeat

In [4]:
def find_min_time_for_epsilon_and_repeats(final_times:np.ndarray,
                                          psi0n:Qobj,
                                          psi1n:Qobj,
                                          H0n:Qobj,
                                          H1n:Qobj,
                                          repeats: int,
                                          epsilon=1 / 10) -> (float, float):
    min_Prepeat = 1 
    old_T = 0
    for T in final_times: 
            _,_, Prepeat = _simulate_time_p_and_pfab_repeat(repeats, T, max(T*5,50) , psi0n, psi1n , H0n, H1n)
#             if Prepeat > old_Prepeat : 
#                 print("BAD: At repeats:{} got Prepeat = {} T = {} total time = {}, epsilon = {} not met in first decrease".
#                       format(repeats,old_Prepeat,old_T,old_T*2*repeats,epsilon))
#                 return 
            if Prepeat <= epsilon : 
                print("GOOD: At repeats:{} got Prepeat = {} T = {} total time = {}".
                      format(repeats,Prepeat,T,T*2*repeats,epsilon))
                return
            min_Prepeat = min(Prepeat,min_Prepeat)
    print("No conclusion for repeats = {} min Prepeat = {}".format(repeats, min_Prepeat))

In [26]:
from concurrent.futures import ThreadPoolExecutor,as_completed
H0n,H1n,psi0n = FDT.prepare_groverlike_system(10)
executor = ThreadPoolExecutor(48)
tasks = []
plt.figure(figsize=(15,15))
for repeats in range(3,100):
    final_times = np.linspace(0.01, 100, 500)
    find_min_time_for_epsilon_and_repeats(final_times, psi0n, basis(2,0) , H0n, H1n, repeats)

No conclusion for repeats = 1 min Prepeat = 0.7538731749937417
No conclusion for repeats = 2 min Prepeat = 0.2578189074712774
GOOD: At repeats:3 got Prepeat = 0.09450563758101764 T = 64.73298597194389 total time = 388.39791583166334
GOOD: At repeats:4 got Prepeat = 0.08507601141506288 T = 39.28462925851703 total time = 314.27703406813623
GOOD: At repeats:5 got Prepeat = 0.08721146612561599 T = 26.66064128256513 total time = 266.6064128256513
GOOD: At repeats:6 got Prepeat = 0.09945159445881925 T = 14.637795591182362 total time = 175.65354709418835
GOOD: At repeats:7 got Prepeat = 0.08495497456225759 T = 13.635891783567132 total time = 190.90248496993985
GOOD: At repeats:8 got Prepeat = 0.07462868257722602 T = 13.034749498997995 total time = 208.5559919839679
GOOD: At repeats:9 got Prepeat = 0.0995243787693501 T = 12.433607214428855 total time = 223.8049298597194


In [11]:
from concurrent.futures import ThreadPoolExecutor,as_completed
H0n,H1n,psi0n = FDT.prepare_groverlike_system(10)
executor = ThreadPoolExecutor(48)
tasks = []
plt.figure(figsize=(15,15))
for repeats in range(11,100):
    final_times = np.linspace(0.01, 12, 500)
    find_min_time_for_epsilon_and_repeats(final_times, psi0n, basis(2,0) , H0n, H1n, repeats)

GOOD: At repeats:11 got Prepeat = 0.09735405095953237 T = 11.831803607214429 total time = 260.2996793587174
GOOD: At repeats:12 got Prepeat = 0.09583103518714192 T = 11.61555110220441 total time = 278.7732264529059
GOOD: At repeats:13 got Prepeat = 0.09803319984646068 T = 11.423326653306614 total time = 297.00649298597193
GOOD: At repeats:14 got Prepeat = 0.09827044726636451 T = 3.710320641282565 total time = 103.88897795591183
GOOD: At repeats:15 got Prepeat = 0.0969761934758722 T = 3.2778156312625253 total time = 98.33446893787576
GOOD: At repeats:16 got Prepeat = 0.09699864953428336 T = 2.965450901803607 total time = 94.89442885771543
GOOD: At repeats:17 got Prepeat = 0.09653440644052097 T = 2.7251703406813625 total time = 92.65579158316632
GOOD: At repeats:18 got Prepeat = 0.09513742528841779 T = 2.532945891783567 total time = 91.18605210420841
GOOD: At repeats:19 got Prepeat = 0.09529423215336598 T = 2.364749498997996 total time = 89.86048096192386
GOOD: At repeats:20 got Prepeat 

In [12]:
from concurrent.futures import ThreadPoolExecutor,as_completed
H0n,H1n,psi0n = FDT.prepare_groverlike_system(11)
executor = ThreadPoolExecutor(48)
tasks = []
plt.figure(figsize=(15,15))
for repeats in range(11,100):
    final_times = np.linspace(0.01, 12, 100)
    find_min_time_for_epsilon_and_repeats(final_times, psi0n, basis(2,0) , H0n, H1n, repeats)

No conclusion for repeats = 11 min Prepeat = 0.3391335104457502
No conclusion for repeats = 12 min Prepeat = 0.26041121397536104
No conclusion for repeats = 13 min Prepeat = 0.18887193128979612
No conclusion for repeats = 14 min Prepeat = 0.12664905514007965
GOOD: At repeats:15 got Prepeat = 0.0755981511391509 T = 12.0 total time = 360.0
GOOD: At repeats:16 got Prepeat = 0.09236582282735341 T = 11.757777777777777 total time = 376.24888888888887
GOOD: At repeats:17 got Prepeat = 0.08346286707901435 T = 11.636666666666667 total time = 395.64666666666665
GOOD: At repeats:18 got Prepeat = 0.07913248491873724 T = 11.515555555555556 total time = 414.56
GOOD: At repeats:19 got Prepeat = 0.09381961179415114 T = 4.127777777777777 total time = 156.85555555555553
GOOD: At repeats:20 got Prepeat = 0.09427651222856122 T = 3.643333333333333 total time = 145.73333333333332
GOOD: At repeats:21 got Prepeat = 0.08554940743335954 T = 3.4011111111111108 total time = 142.84666666666666
GOOD: At repeats:22 

In [13]:
from concurrent.futures import ThreadPoolExecutor,as_completed
H0n,H1n,psi0n = FDT.prepare_groverlike_system(12)
executor = ThreadPoolExecutor(48)
tasks = []
plt.figure(figsize=(15,15))
for repeats in range(20,100):
    final_times = np.linspace(0.01, 12, 100)
    find_min_time_for_epsilon_and_repeats(final_times, psi0n, basis(2,0) , H0n, H1n, repeats)

No conclusion for repeats = 20 min Prepeat = 0.11536291224557782
GOOD: At repeats:21 got Prepeat = 0.07940863037476478 T = 12.0 total time = 504.0
GOOD: At repeats:22 got Prepeat = 0.07701139592186605 T = 11.87888888888889 total time = 522.6711111111111
GOOD: At repeats:23 got Prepeat = 0.07768522219961846 T = 11.757777777777777 total time = 540.8577777777778
GOOD: At repeats:24 got Prepeat = 0.08148377149855301 T = 11.636666666666667 total time = 558.56
GOOD: At repeats:25 got Prepeat = 0.08857786411849246 T = 11.515555555555556 total time = 575.7777777777778
GOOD: At repeats:26 got Prepeat = 0.09733361012622439 T = 4.612222222222222 total time = 239.83555555555554
GOOD: At repeats:27 got Prepeat = 0.09567868559943338 T = 4.006666666666667 total time = 216.36
GOOD: At repeats:28 got Prepeat = 0.09953089538336633 T = 3.643333333333333 total time = 204.02666666666664
GOOD: At repeats:29 got Prepeat = 0.08659928514611494 T = 3.522222222222222 total time = 204.2888888888889
GOOD: At repea

KeyboardInterrupt: 

In [14]:
from concurrent.futures import ThreadPoolExecutor,as_completed
H0n,H1n,psi0n = FDT.prepare_groverlike_system(13)
executor = ThreadPoolExecutor(48)
tasks = []
plt.figure(figsize=(15,15))
for repeats in range(40,100):
    final_times = np.linspace(0.01, 12, 100)
    find_min_time_for_epsilon_and_repeats(final_times, psi0n, basis(2,0) , H0n, H1n, repeats)

GOOD: At repeats:40 got Prepeat = 0.09047529364728511 T = 3.643333333333333 total time = 291.46666666666664
GOOD: At repeats:41 got Prepeat = 0.09935807057294638 T = 3.4011111111111108 total time = 278.89111111111106
GOOD: At repeats:42 got Prepeat = 0.09724277118639339 T = 3.28 total time = 275.52
GOOD: At repeats:43 got Prepeat = 0.09706168314002464 T = 3.158888888888889 total time = 271.6644444444444
GOOD: At repeats:44 got Prepeat = 0.09886012772481648 T = 3.0377777777777775 total time = 267.3244444444444
GOOD: At repeats:45 got Prepeat = 0.08254656595980404 T = 3.0377777777777775 total time = 273.4
GOOD: At repeats:46 got Prepeat = 0.08655682898605634 T = 2.9166666666666665 total time = 268.3333333333333
GOOD: At repeats:47 got Prepeat = 0.0926836765812014 T = 2.7955555555555556 total time = 262.78222222222223
GOOD: At repeats:48 got Prepeat = 0.07771283991938037 T = 2.7955555555555556 total time = 268.37333333333333


KeyboardInterrupt: 

In [17]:
from concurrent.futures import ThreadPoolExecutor,as_completed
H0n,H1n,psi0n = FDT.prepare_groverlike_system(14)
executor = ThreadPoolExecutor(48)
tasks = []
plt.figure(figsize=(15,15))
for repeats in range(50,100):
    final_times = np.linspace(0.01, 12, 100)
    find_min_time_for_epsilon_and_repeats(final_times, psi0n, basis(2,0) , H0n, H1n, repeats)

GOOD: At repeats:50 got Prepeat = 0.08505334860089687 T = 11.515555555555556 total time = 1151.5555555555557
GOOD: At repeats:51 got Prepeat = 0.07139933217125721 T = 11.515555555555556 total time = 1174.5866666666668
GOOD: At repeats:52 got Prepeat = 0.09733725091020386 T = 4.491111111111111 total time = 467.0755555555555
GOOD: At repeats:53 got Prepeat = 0.09903102094948629 T = 4.127777777777777 total time = 437.5444444444444
GOOD: At repeats:54 got Prepeat = 0.09290949754072454 T = 4.006666666666667 total time = 432.72
GOOD: At repeats:55 got Prepeat = 0.09852980054331988 T = 3.7644444444444445 total time = 414.0888888888889
GOOD: At repeats:56 got Prepeat = 0.09673995195858845 T = 3.643333333333333 total time = 408.0533333333333
GOOD: At repeats:57 got Prepeat = 0.09654222552426373 T = 3.522222222222222 total time = 401.5333333333333
GOOD: At repeats:58 got Prepeat = 0.09796880613608645 T = 3.4011111111111108 total time = 394.52888888888884
GOOD: At repeats:59 got Prepeat = 0.08550

In [18]:
from concurrent.futures import ThreadPoolExecutor,as_completed
H0n,H1n,psi0n = FDT.prepare_groverlike_system(15)
executor = ThreadPoolExecutor(48)
tasks = []
plt.figure(figsize=(15,15))
for repeats in range(60,100):
    final_times = np.linspace(0.01, 12, 100)
    find_min_time_for_epsilon_and_repeats(final_times, psi0n, basis(2,0) , H0n, H1n, repeats)

GOOD: At repeats:60 got Prepeat = 0.09863618813978536 T = 11.87888888888889 total time = 1425.4666666666667
GOOD: At repeats:61 got Prepeat = 0.08656061281651649 T = 11.87888888888889 total time = 1449.2244444444445
GOOD: At repeats:62 got Prepeat = 0.07520469710671204 T = 11.87888888888889 total time = 1472.9822222222224
GOOD: At repeats:63 got Prepeat = 0.09612059349776801 T = 11.757777777777777 total time = 1481.48
GOOD: At repeats:64 got Prepeat = 0.08470450219293096 T = 11.757777777777777 total time = 1504.9955555555555
GOOD: At repeats:65 got Prepeat = 0.07394857182572757 T = 11.757777777777777 total time = 1528.511111111111
GOOD: At repeats:66 got Prepeat = 0.09724155041596637 T = 11.636666666666667 total time = 1536.04
GOOD: At repeats:67 got Prepeat = 0.08628746902637552 T = 11.636666666666667 total time = 1559.3133333333333
GOOD: At repeats:68 got Prepeat = 0.07593081403181454 T = 11.636666666666667 total time = 1582.5866666666666
GOOD: At repeats:69 got Prepeat = 0.066186537

In [22]:
from concurrent.futures import ThreadPoolExecutor,as_completed
H0n,H1n,psi0n = FDT.prepare_groverlike_system(16)
executor = ThreadPoolExecutor(48)
tasks = []
plt.figure(figsize=(15,15))
for repeats in range(110,120):
    final_times = np.linspace(0.01, 12, 100)
    find_min_time_for_epsilon_and_repeats(final_times, psi0n, basis(2,0) , H0n, H1n, repeats)

GOOD: At repeats:110 got Prepeat = 0.09710491842068818 T = 3.7644444444444445 total time = 828.1777777777778
GOOD: At repeats:111 got Prepeat = 0.0904596367221363 T = 3.7644444444444445 total time = 835.7066666666667
GOOD: At repeats:112 got Prepeat = 0.09532405587555672 T = 3.643333333333333 total time = 816.1066666666666
GOOD: At repeats:113 got Prepeat = 0.08883528404925238 T = 3.643333333333333 total time = 823.3933333333333
GOOD: At repeats:114 got Prepeat = 0.09512760518929561 T = 3.522222222222222 total time = 803.0666666666666
GOOD: At repeats:115 got Prepeat = 0.08875524846587254 T = 3.522222222222222 total time = 810.1111111111111
GOOD: At repeats:116 got Prepeat = 0.09654758144081454 T = 3.4011111111111108 total time = 789.0577777777777
GOOD: At repeats:117 got Prepeat = 0.09025273414221614 T = 3.4011111111111108 total time = 795.8599999999999
GOOD: At repeats:118 got Prepeat = 0.09964972253203842 T = 3.28 total time = 774.0799999999999
GOOD: At repeats:119 got Prepeat = 0.0